In [4]:
import random
import heapq
from numpy.linalg import cholesky
import numpy as np
from collections import deque
from tetris_utils import *
from tetris_reinforcement_learner import TetrisReinforcementLearner
from tetris import *
from copy import deepcopy
import pygame,sys
import os
import matplotlib.pyplot as plt
from tetris import TetrisApp
from tetris_reinforcement_learner import TetrisReinforcementLearner
App=TetrisApp()
RL_learner=TetrisReinforcementLearner(App)


## If you want, you can skip to the last cell to see the performance :)

## some functions

In [5]:
RL_learner=TetrisReinforcementLearner(App)
App=TetrisApp()
done = False
batch_size = 40


state_size = 270
action_size = 30
# agent.load("./save/cartpole-dqn.h5")
App.init_game()
def pixalize_stone(stone):
    container=np.zeros((2,10))
    if len(stone)==1:
        container[0][...,3:3+len(stone[0])]=stone[0]
    elif stone[0][0]==7:
        container[0][...,4:4+len(stone[0])]=stone[0]
        container[1][...,4:4+len(stone[0])]=stone[1]
    else:
        container[0][...,3:3+len(stone[0])]=stone[0]
        container[1][...,3:3+len(stone[0])]=stone[1]
    return container
    
def preprocess_data(state):
    
    board=np.array(state['board'])
    stone=state['stone']
    next_stone=state['next_stone']
    pixal_stone=pixalize_stone(stone)
    pixal_next_stone=pixalize_stone(next_stone)

    final_board=np.append(pixal_next_stone,pixal_stone,axis=0)
    final_board=np.append(final_board,board,axis=0)
    final_board=np.where(final_board>0,1,0)
#     print(final_board)
    final_board=final_board.flatten()
    final_board=final_board.reshape(1,270)
    return final_board


def update_new(reinforcement_learner, old_state,new_state):## Taking holes into account
    change_in_pile_height = reinforcement_learner.get_pile_height(new_state) - reinforcement_learner.get_pile_height(old_state)
    reward = -1 * change_in_pile_height
    return reward# container=np.zeros((len(stone),10))
# for i in range(len(stone)):
#     container[i][...,3:3+len(stone[0])]=stone[i]
# board=board.flatten()
# board=board[:,np.newaxis]
# board = np.where(board>0,1,0)
# stone_idx=find_stone_idx(stone)
# next_stone_idx=find_stone_idx(state['next_stone'])

def get_feature(state):
    height=RL_learner.get_pile_height(state)
    holes=RL_learner.get_holes(state)
    contours=RL_learner.get_contours(state)
    row_holes=RL_learner.get_row_holes(state)
    features=np.array([[height],[holes],[contours],[row_holes]])
    return features
def get_current_state_CEM_value(state,action_sequence):
    successor_state=RL_learner.get_successor_state(state,action_sequence)
    features=get_feature(successor_state)
    for feature, value in features.items():
        CEM_val+=RL_learner.CEM_weights[feature]*value
        
def get_following_state_CEM_value(state,action_sequence,cur_theta):
    
    successor_state=RL_learner.get_successor_state(state,action_sequence)
    next_state_reward=RL_learner.get_next_action_reward(state,action_sequence)
#     print('next_state_reward is',next_state_reward)
    done=successor_state['gameover']
    if not done:
        
        features=get_feature(successor_state)
        features=np.append(features,np.array([[next_state_reward]]),axis=0)
    return cur_theta.T.dot(features)

def get_sample_theta(mu,sigma,num_sample):
    R = cholesky(sigma)
    s = np.dot(np.random.randn(num_sample, 5), R) + mu.T
    return s


## initialize stuff, which has been done. The latest theta is store in the 'top_15_theta_75samples.txt', file

## if you want to search the theta from the beginning, you can uncomment the following code, and then run the next cell.

In [6]:
# top_15_theta = deque(maxlen=15)
# mu=np.zeros((5,1)) #initialize mu
# sigma=100*np.eye(5,5) #initialize theta
# s=get_sample_theta(mu,sigma,100)
# num_sample=50
# theta_grade=[] # store the grade each theta earned
# for theta_idx in range(num_sample):

#     App.init_game()
#     if not App.pygame_initted: App.init_pygame()
#     App.gameover=False
#     App.paused=False
#     App.start_game()
#     done=False
#     while not done:
#         App.display_board()
#         state=RL_learner.capture_state_attributes(App)
#         dont_burn_my_cpu=pygame.time.Clock()  

#         cur_theta=np.array(s[theta_idx])
#         cur_theta=cur_theta[:,np.newaxis]

#         ## calculate the next state value, which is store in the list: next_val
#         next_val=[] # store the next_state_value
#         legal_action_sequences=RL_learner.get_legal_action_sequences(state)
#         for actions in legal_action_sequences:
#             next_val.append(get_following_state_CEM_value(state,actions,cur_theta))
#         next_action=legal_action_sequences[np.argmax(next_val)]
#         App.play_action_sequence(next_action)
#         new_state=RL_learner.capture_state_attributes(App)
#         done=new_state['gameover']
#         if done:
#             theta_grade.append(App.lines)

# top_15_theta_idx=list(map(theta_grade.index, heapq.nlargest(15, theta_grade)))
# for idx in top_15_theta_idx:
#     top_15_theta.append(s[idx])
# statistic1.append([np.max(theta_grade),np.min(theta_grade),np.mean(theta_grade)])
# print(statistic1)
# statistic_np=np.array(statistic1)

## Load the latest prameter theta from the file

In [7]:
top_15_load_theta= np.loadtxt('top_15_theta_75samples.txt')
statistic1=[]
top_15_theta = deque(maxlen=15)
top_15_theta=top_15_load_theta
top_15_theta=top_15_theta.tolist()

In [10]:

for i in range(10000):
    np_top_15_theta=np.array(top_15_theta).T
    mu=np.mean(np_top_15_theta,axis=1)
    sigma=np.cov(np_top_15_theta)+np.eye(5)*1e-16#+max(5-(i+1)/10,0)*np.eye(3)
    s=get_sample_theta(mu,sigma,100)

    num_sample=75
    theta_grade=[] # store the grade each theta earned
    for theta_idx in range(num_sample):
        print('the theta_idx is', theta_idx)
        App.init_game()
        if not App.pygame_initted: App.init_pygame()
        App.gameover=False
        App.paused=False
        App.start_game()
        done=False
        cur_theta=np.array(s[theta_idx])
        cur_theta=cur_theta[:,np.newaxis]
        while not done:
            App.display_board()
            state=RL_learner.capture_state_attributes(App)
            dont_burn_my_cpu=pygame.time.Clock()  
            ## calculate the next state value, which is store in the list: next_val
            next_val=[] # store the next_state_value
            legal_action_sequences=RL_learner.get_legal_action_sequences(state)
            for actions in legal_action_sequences:
                next_val.append(get_following_state_CEM_value(state,actions,cur_theta))
            next_action=legal_action_sequences[np.argmax(next_val)]
            App.play_action_sequence(next_action)
            new_state=RL_learner.capture_state_attributes(App)
            done=new_state['gameover']
            if done:
                theta_grade.append(App.lines)

    top_15_theta_idx=list(map(theta_grade.index, heapq.nlargest(15, theta_grade)))
    for idx in top_15_theta_idx:
        top_15_theta.append(s[idx])
        
    statistic1.append([np.max(theta_grade),np.min(theta_grade),np.mean(theta_grade)])
#     print(i)
#     print(statistic1)
    statistic_np=np.array(statistic1)
    np.savetxt("statistics_of_5_feature_without_noise.txt",statistic_np)
    distribution=np.array(top_15_theta)
    np.savetxt("top_15_theta_75samples.txt",distribution)

## here is the best theta we get:

theta:  [[-0.3645792 ,
        -28.55589636,  
        -5.72603727,  
        -1.3392655 ,
        4.34739603]] which leads to 4397 line cleared
        
theta: [[0.19267331,
       -27.34949633, 
       -5.62961297,  
       -1.33885247,
       4.34739604]] which leads to 9044 line cleared
                
statistic              
[[4397, 15, 829.8666666666667],
 [4397, 15, 829.8666666666667],
 [9044, 18, 789.9866666666667]]

## You can try out the following  code to see the performance:

In [25]:
cur_theta=np.array([0.19267331,
       -27.34949633, 
       -5.62961297,  
       -1.33885247,
       4.34739604])

In [28]:
for i in range(10):
    App.init_game()
    if not App.pygame_initted: App.init_pygame()
    App.gameover=False
    App.paused=False
    App.start_game()
    done=False
    cur_theta=np.array(cur_theta)
    cur_theta=cur_theta[:,np.newaxis]
    while not done:
        App.display_board()
        state=RL_learner.capture_state_attributes(App)
        dont_burn_my_cpu=pygame.time.Clock()  
        ## calculate the next state value, which is store in the list: next_val
        next_val=[] # store the next_state_value
        legal_action_sequences=RL_learner.get_legal_action_sequences(state)
        for actions in legal_action_sequences:
            next_val.append(get_following_state_CEM_value(state,actions,cur_theta))
        next_action=legal_action_sequences[np.argmax(next_val)]
        App.play_action_sequence(next_action)
        new_state=RL_learner.capture_state_attributes(App)
        done=new_state['gameover']
        if done:
            theta_grade.append(App.lines)

KeyboardInterrupt: 